In [3]:
from langgraph.types import Command

In [5]:
import pandas as pd
df = pd.read_excel("정보성키워드_20240229.xlsx")


In [15]:
df["월간검색수"] = df["월간검색수_PC"] + df["월간검색수_모바일"]
df = df.sort_values(by="월간검색수", ascending=False)
df['연관키워드'].iloc[0]

'오늘의운세'

In [ ]:
from datetime import datetime
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

create_react_agent

In [41]:
# 네이버 API 인증 정보 (본인의 정보로 교체)
CLIENT_ID = "8YrCNtRhkQNIQE26fR9B"
CLIENT_SECRET = "oMVEExn99m"

In [40]:
naver_search(query='유팡', search_type='shop')

{'lastBuildDate': 'Sun, 09 Mar 2025 18:19:05 +0900',
 'total': 1010,
 'start': 1,
 'display': 10,
 'items': [{'title': '<b>유팡</b> 시그니처2 젖병소독기 UP920A',
   'link': 'https://search.shopping.naver.com/catalog/49780621507',
   'image': 'https://shopping-phinf.pstatic.net/main_4978062/49780621507.20241118175157.jpg',
   'lprice': '342930',
   'hprice': '',
   'mallName': '네이버',
   'productId': '49780621507',
   'productType': '1',
   'brand': '유팡',
   'maker': '유팡',
   'category1': '출산/육아',
   'category2': '소독/살균용품',
   'category3': '젖병소독기',
   'category4': ''},
  {'title': '[출산완벽준비] 시그니처2 UP920A+악세서리 3종세트',
   'link': 'https://smartstore.naver.com/main/products/11095987201',
   'image': 'https://shopping-phinf.pstatic.net/main_8864049/88640497523.jpg',
   'lprice': '409000',
   'hprice': '',
   'mallName': '유팡UPANG',
   'productId': '88640497523',
   'productType': '2',
   'brand': '유팡',
   'maker': '',
   'category1': '출산/육아',
   'category2': '소독/살균용품',
   'category3': '젖병소독기',
   'categor

In [3]:
import os
import urllib.request
import urllib.parse
import json
from typing import Dict, List, TypedDict, Any, Optional
from dotenv import load_dotenv

# ----- langchain + openai_functions 관련 임포트
from langchain.chains.openai_functions import create_structured_output_runnable
# (환경에 맞게 langchain 관련 패키지를 교체하세요)
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

load_dotenv()  # .env에서 NAVER_CLIENT_ID, NAVER_CLIENT_SECRET 로드

CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

# --------------------
# 1) TypedDict 정의
# --------------------
class KeywordType(TypedDict):
    """키워드 분류 결과 타입"""
    keyword: str
    type: str           # 'brand', 'information', 'product'
    confidence: float
    analysis: str
    category: Optional[List[str]]  # 상품 키워드인 경우, [고빈도 category1, category2, category3, category4]

# --------------------
# 2) 네이버 쇼핑 검색 함수
# --------------------
def naver_search(query: str, search_type: str = "shop") -> Dict[str, Any]:
    """
    네이버 API를 사용하여 검색 결과를 가져옵니다.
    """
    if not CLIENT_ID or not CLIENT_SECRET:
        raise ValueError("NAVER_CLIENT_ID와 NAVER_CLIENT_SECRET을 환경 변수에 설정해야 합니다.")
    
    enc_text = urllib.parse.quote(query)
    url = f"https://openapi.naver.com/v1/search/{search_type}.json?query={enc_text}"
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", CLIENT_ID)
    request.add_header("X-Naver-Client-Secret", CLIENT_SECRET)
    
    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if rescode == 200:
            return json.loads(response.read().decode('utf-8'))
        else:
            print("Error Code:", rescode)
            return {}
    except Exception as e:
        print("검색 중 예외 발생:", e)
        return {}

# --------------------
# 3) 브랜드 일관성 분석
# --------------------
def analyze_brand_consistency(search_results: Dict[str, Any]) -> Dict[str, Any]:
    """
    검색 결과에서 브랜드/제조사의 일관성을 분석합니다.
    """
    if not search_results or "items" not in search_results:
        return {"is_consistent": False, "analysis": "검색 결과가 없습니다.", "analysis_detail": {}}
    
    items = search_results.get("items", [])
    if not items:
        return {"is_consistent": False, "analysis": "검색 결과 아이템이 없습니다.", "analysis_detail": {}}
    
    # 브랜드/제조사 정보 추출
    brands = [item.get("brand", "").strip() for item in items if "brand" in item]
    makers = [item.get("maker", "").strip() for item in items if "maker" in item]
    
    valid_brands = [b for b in brands if b]
    valid_makers = [m for m in makers if m]
    
    unique_brands = set(valid_brands)
    unique_makers = set(valid_makers)
    
    brand_counts = {brand: valid_brands.count(brand) for brand in unique_brands}
    maker_counts = {maker: valid_makers.count(maker) for maker in unique_makers}
    
    # 브랜드/메이커 중 가장 많이 등장한 횟수 계산
    brand_consistency_ratio = 0.0
    if valid_brands:
        most_freq_brand_count = max(brand_counts.values())
        brand_consistency_ratio = most_freq_brand_count / len(valid_brands)
    
    maker_consistency_ratio = 0.0
    if valid_makers:
        most_freq_maker_count = max(maker_counts.values())
        maker_consistency_ratio = most_freq_maker_count / len(valid_makers)
    
    # 80% 이상의 상품이 동일 브랜드 또는 maker + 충분한 아이템 수가 있어야 브랜드 일관성 있음으로 판단
    is_consistent = False
    if (brand_consistency_ratio >= 0.8 and len(valid_brands) >= 5) or \
       (maker_consistency_ratio >= 0.8 and len(valid_makers) >= 5):
        is_consistent = True
    
    analysis_detail = {
        "total_items": len(items),
        "valid_brand_count": len(valid_brands),
        "valid_maker_count": len(valid_makers),
        "brand_counts": brand_counts,
        "maker_counts": maker_counts,
        "brand_consistency_ratio": brand_consistency_ratio,
        "maker_consistency_ratio": maker_consistency_ratio,
    }
    return {
        "is_consistent": is_consistent,
        "analysis": f"브랜드 일관성 {is_consistent}",
        "analysis_detail": analysis_detail
    }

# --------------------
# 4) 각 category 필드별 고빈도 카테고리 추출 함수
# --------------------
def extract_high_frequency_categories(search_results: Dict[str, Any]) -> List[str]:
    """
    네이버 쇼핑 검색 결과에서 각 category1 ~ category4 필드의 고빈도(최다 등장) 값을 추출합니다.
    결과는 [고빈도 category1, 고빈도 category2, 고빈도 category3, 고빈도 category4] 형식의 리스트로 반환됩니다.
    """
    high_freq_categories = []
    for key in ['category1', 'category2', 'category3', 'category4']:
        freq = {}
        for item in search_results.get("items", []):
            value = item.get(key, "").strip()
            if value:
                freq[value] = freq.get(value, 0) + 1
        if freq:
            most_freq = max(freq, key=freq.get)
            high_freq_categories.append(most_freq)
        else:
            high_freq_categories.append("")
    return high_freq_categories

# --------------------
# 5) LLM으로 '상품 vs 정보성' 분류하기
#    - 브랜드가 아닌 경우에만 활용
# --------------------
class LLMClassificationResult(BaseModel):
    """LLM이 최종 분류를 반환"""
    classification: str = Field(..., description="분류 결과. '상품' 또는 '정보성' 중 하나를 예상")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "당신은 네이버 쇼핑검색 결과에 대한 전문 분석가입니다. "
        "입력으로 {keyword}가 주어집니다. "
        "단, 이미 '브랜드'가 아님은 확정된 상태이므로, "
        "'상품' 혹은 '정보성' 중 하나를 한국어로 결정해 주세요."
    ),
    (
        "human",
        "검색결과를 분석하여 '{keyword}'가 '상품'인지 '정보성'인지 판단해 주세요. "
        "단, 브랜드는 이미 제외된 상태입니다."
    ),
])
llm_chain = prompt | llm.with_structured_output(LLMClassificationResult)

def llm_classify_product_or_info(keyword: str) -> str:
    """
    LLM을 이용해 '상품' vs '정보성' 분류
    """
    try:
        result = llm_chain.invoke({"keyword": keyword})
        return result.classification
    except Exception as e:
        print("LLM 분류 에러:", e)
        return "정보성"

# --------------------
# 6) 최종 분류 함수
#    - (1) 브랜드 분석 -> 일관성 있으면 'brand'
#    - (2) 브랜드가 아니면 LLM으로 '상품' vs '정보성' 판별
#    - 상품성 키워드인 경우, 고빈도 category 리스트 추가 (정보성은 비워둠)
# --------------------
def classify_keyword(keyword: str) -> KeywordType:
    """
    주어진 키워드가 'brand', 'information', 'product' 중 어느 유형인지 최종 분류합니다.
    상품성 키워드인 경우에는 네이버 쇼핑 검색 결과에서 각 category 필드별 고빈도 값을 추출하여 리스트로 추가합니다.
    만약 type이 information이면 category 필드는 비워둡니다.
    """
    try:
        # 1) 네이버 쇼핑 검색
        blog_res = naver_search(keyword, "blog")
        shop_res = naver_search(keyword, "shop")
        if not shop_res or "items" not in shop_res or not shop_res["items"]:
            # 검색 결과 없으면 blog 검색 진행
            
            blog_cnt = len(blog_res["items"]) if blog_res and "items" in blog_res else 0
            
            if blog_cnt > 0:
                return {
                    "keyword": keyword,
                    "type": "information",
                    "confidence": 0.7,
                    "analysis": f"쇼핑 검색 결과가 없고, 블로그 결과 {blog_cnt}개 => 정보성 키워드로 추정",
                    "category": None
                }
            else:
                return {
                    "keyword": keyword,
                    "type": "unknown",
                    "confidence": 0.0,
                    "analysis": "검색 결과가 전혀 없습니다.",
                    "category": None
                }
        
        # 2) 브랜드 일관성 분석
        brand_analysis = analyze_brand_consistency(shop_res)
        if brand_analysis["is_consistent"]:
            detail = brand_analysis["analysis_detail"]
            confidence = max(detail["brand_consistency_ratio"], detail["maker_consistency_ratio"])
            confidence = min(confidence + 0.1, 1.0)  # 약간 상승 조정
            
            # 브랜드로 분류하는 경우는 상품 카테고리 리스트를 추가하지 않음 (혹은 별도 처리 가능)
            return {
                "keyword": keyword,
                "type": "brand",
                "confidence": confidence,
                "analysis": f"브랜드 일관성 {confidence*100:.1f}%. 계량적으로 브랜드로 판단",
                "category": None,
                "titles": [i['title'] for i in blog_res['items']],
                "description": [i['description'] for i in blog_res['items']],
            }
        else:
            # 브랜드가 아니라면 LLM으로 '상품' vs '정보성' 판별
            classification = llm_classify_product_or_info(keyword)
            
            if classification == "상품":
                # 상품성 키워드: 네이버 쇼핑 결과에서 각 category 필드별 고빈도 값 추출
                high_freq_categories = extract_high_frequency_categories(shop_res)
                return {
                    "keyword": keyword,
                    "type": "product",
                    "confidence": 0.8,  # 임의로 0.8 부여
                    "analysis": "LLM 분류 결과: 상품",
                    "category": high_freq_categories,
                    "titles": [i['title'] for i in blog_res['items']],
                    "description": [i['description'] for i in blog_res['items']],   
                }
            else:
                return {
                    "keyword": keyword,
                    "type": "information",
                    "confidence": 0.8,
                    "analysis": "LLM 분류 결과: 정보성",
                    "category": None,
                    "titles": [i['title'] for i in blog_res['items']],
                    "description": [i['description'] for i in blog_res['items']],
                }
    except Exception as e:
        import traceback
        return {
            "keyword": keyword,
            "type": "unknown",
            "confidence": 0.0,
            "analysis": f"분류 중 예외 발생: {str(e)}\n{traceback.format_exc()}",
            "category": None,
            "titles": [i['title'] for i in blog_res['items']],
            "description": [i['description'] for i in blog_res['items']],
        }

# --------------------
# 7) 테스트
# --------------------
if __name__ == "__main__":
    test_keywords = ["골밀도", "프리미엄견과류", "어린이날여행", "유팡", "MIMAXARI"]  
    for kw in test_keywords:
        print(f"\n===== 키워드 '{kw}' 분석 =====")
        result = classify_keyword(kw)
        print(f"유형: {result['type']}")
        print(f"신뢰도: {result['confidence']:.2f}")
        print(f"분석: {result['analysis']}")
        print(f"카테고리: {result['category']}")
        print(f"titles: {result['titles']}")
        print(f"description: {result['description']}")


c:\Users\PC_1M\anaconda3\envs\py312\Lib\site-packages\langchain_openai\chat_models\base.py:1377: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(



===== 키워드 '골밀도' 분석 =====
유형: information
신뢰도: 0.80
분석: LLM 분류 결과: 정보성
카테고리: None
titles: ['<b>골밀도</b> 영양제 천연칼슘이 있다? 골다공증예방후기!(2025... ', '<b>골밀도</b> 이어폰 귀가 편안한 제품', '심플체크업검사 후기(30대 여자/초음파,CT,<b>골밀도</b> 등)', '<b>골밀도</b> 영양제 덕분에 효도한 딸', '<b>골밀도</b> 영양제 부모님선물로 딱!', '귀 건강을 위해 선택한 <b>골밀도</b> 이어폰!', '칼슘제효능 좋은 걸로 챙겨 주며 <b>골밀도</b>를 높여 보아요!', '골다공증 주사 심각한 부작용 <b>골밀도</b> 증가시키는 방법... ', '칼슘마그네슘영양제로 <b>골밀도</b> 수치 관리 시작', '<b>골밀도</b> 정상수치 범위 높이는 관리 방법']
description: ['<b>골밀도</b> 영양제를 찾고 계시다면 결정을 하기 전 꼭 이 글을 읽어보고 선택을 하시기 바랍니다. 이... <b>골밀도</b> 낮아지면 나타나는 증상 2. 골다공증예방 천연칼슘은? 3. 골다공증 영양제 중요하게 따져봐야... ', '새로 구입해야 하나 고민하다가 요즘 <b>골밀도</b> 이어폰이 좋다길래 써보고 싶어서 알아보게 됐어요. ”... <b>골밀도</b> 이어폰은 두 가지 종류의 장점을 살려 만든 제품이더라고요. 일단 무선이라 선이 꼬일 걱정이 없고... ', '분당차병원 국가검진+개인특화 심플체크업검사 후기(30대 여자/초음파,CT,<b>골밀도</b> 등) 방금... 싶었던 <b>골밀도</b>, 복부초음파검사가 있었고 단품으로 받는것보단 저게 나은것 같아 심플업 선택했다 ㅎㅎ 흉부ct는... ', '조금이나마 증상을 편하게 해드리려 <b>골밀도</b> 영양제 후레오스를 구입했어요. 주변 친구들보다 빨리... 다양한 검사를 한 선생님이 엄마의 연세를 확인하시더니 <b>골밀도</b> 수치가 낮다고 하셨죠. 폐경을 맞이한... ', '요즘

In [4]:
from keyword_classifier import llm_generate_title

c:\Users\PC_1M\anaconda3\envs\py312\Lib\site-packages\langchain_openai\chat_models\base.py:1377: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [7]:
generated_title = llm_generate_title(result)

In [8]:
generated_title

'미마자리 유모차, 부모가 선택한 최고의 선택!'

In [30]:
from langgraph.checkpoint.memory import MemorySaver
from src.graph import builder
from langgraph.types import Command
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

import uuid 
thread = {"configurable": {"thread_id": str(uuid.uuid4()),
                           "search_api": "tavily", # perplexity, exa, arxiv, pubmed, linkup
                           "planner_provider": "openai",# 
                           "planner_model": "gpt-4o-mini",
                           "writer_provider": "openai",
                           "writer_model": "gpt-4o-mini",
                           "max_search_depth": 2,
                           }}

topic = '미마자리 유모차, 부모가 선택한 최고의 선택!'
async for event in graph.astream({"topic":topic,}, thread, stream_mode="updates"):
    print(event)
    print("\n")
    
# When you are satisfied with the report plan, you can pass True to proceed to report generation:
# Pass True to approve the report plan and proceed to report generation
events = []
async for event in graph.astream(Command(resume=True), thread, stream_mode="updates"):
    events.append(event)
    print(event)
    print("\n")


{'generate_report_plan': {'sections': [Section(name='Introduction', description='A brief overview of the topic highlighting the significance of the Mima stroller as a popular choice among parents.', research=False, content=''), Section(name='Design and Features', description='An overview of the design aesthetics and practical features of the Mima stroller, including the high-end materials used and the unique folding mechanisms.', research=True, content=''), Section(name='Safety Considerations', description='A section focused on the safety features of the Mima stroller, discussing aspects such as stability, harnesses, and safety ratings.', research=True, content=''), Section(name='Usability and Comfort', description='An exploration of the usability aspects, including maneuverability, comfort for both parent and child, and storage capacity.', research=True, content=''), Section(name='Comparative Analysis', description='A comparison of the Mima stroller with other premium stroller brands 

In [4]:


from IPython.display import HTML
HTML(event['compile_final_report']['final_report'])

In [ ]:
event['compile_final_report']['final_report']

In [ ]:
from markdown_to_html_converter import convert_markdown_to_html
html_output = convert_markdown_to_html(event['compile_final_report']['final_report'])

print(html_output)

# 결과를 파일로 저장
with open("output.html", "w", encoding="utf-8") as f:
    f.write(html_output) 
